<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto, CA</font></h1>

## Introduction

In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup #library for scraping from web

!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

<a id='item1'></a>

## 1. Download and Explore Dataset

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

soup = BeautifulSoup(page.content,'html.parser')

Get the Table 

In [4]:
table = soup.find('table').prettify()
table

'<table class="wikitable sortable">\n <tbody>\n  <tr>\n   <th>\n    Postcode\n   </th>\n   <th>\n    Borough\n   </th>\n   <th>\n    Neighbourhood\n   </th>\n  </tr>\n  <tr>\n   <td>\n    M1A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M2A\n   </td>\n   <td>\n    Not assigned\n   </td>\n   <td>\n    Not assigned\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M3A\n   </td>\n   <td>\n    <a href="/wiki/North_York" title="North York">\n     North York\n    </a>\n   </td>\n   <td>\n    <a href="/wiki/Parkwoods" title="Parkwoods">\n     Parkwoods\n    </a>\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M4A\n   </td>\n   <td>\n    <a href="/wiki/North_York" title="North York">\n     North York\n    </a>\n   </td>\n   <td>\n    <a href="/wiki/Victoria_Village" title="Victoria Village">\n     Victoria Village\n    </a>\n   </td>\n  </tr>\n  <tr>\n   <td>\n    M5A\n   </td>\n   <td>\n    <a href="/wiki/Downtown_Toronto" title="Downtown 

#### Tranform the data into a *pandas* dataframe

In [5]:
df = pd.read_html(table, match='str', header=0) # Returns a list of dataframes from tables tags in the input string. header=0 is denote the first row conatins the column lables.
table_df = df[0] # First dataframe in the list
table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 3 columns):
Postcode         289 non-null object
Borough          289 non-null object
Neighbourhood    289 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [6]:
table_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Removing the Borough that have "Not assigned"

In [7]:
table_df2 = table_df[table_df.Borough != 'Not assigned'].reset_index(drop=True)
table_df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
table_df2 = table_df2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
table_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


#### Assigning Neighbourhood to Borough for Not Assigned recods in Neighbourhood

In [9]:
table_df2.loc[table_df2.Neighbourhood=='Not assigned'].reset_index(drop=True)

,Postcode,Borough,Neighbourhood
0,M7A,Queen's Park,Not assigned


In [10]:
table_df2.loc[table_df2.Neighbourhood=='Not assigned', 'Neighbourhood']=table_df2.Borough
table_df2.loc[table_df2.Neighbourhood=='Not assigned'].reset_index(drop=True) # Should have no rows satisfying this condition excists
table_df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Number of Rows in dataframe

In [11]:
print('Number of Rows =', table_df2.shape[0])

Number of Rows = 103


## Part 2 - Getting the Latitude and Longitude

In [12]:
table_df2['Latitude']=''
table_df2['Longitude']=''

In [13]:
cord_df = pd.read_csv("http://cocl.us/Geospatial_data")
cord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
table_df2 = table_df2.rename(columns={'Postcode':'PostalCode'})
table_df2.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",,
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",,
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


In [15]:
for i in range(len(cord_df)):
    table_df2.at[table_df2['PostalCode']==cord_df.iloc[i]['Postal Code'],'Latitude']=cord_df.iloc[i]['Latitude']
    table_df2.at[table_df2['PostalCode']==cord_df.iloc[i]['Postal Code'],'Longitude']=cord_df.iloc[i]['Longitude']

In [16]:
table_df2

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.6927,-79.2648


## Part 3 - Clustering Neighbourhood

In [17]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.7170226, -79.4197830350134.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(table_df2['Latitude'], table_df2['Longitude'], table_df2['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
CLIENT_ID = 'XHHVYHJ5Z54HY4E0YA5SLUVBJPDCXWJBDTS1SC5BYHR3C0YP' # your Foursquare ID
CLIENT_SECRET = 'COCN0HX03VMYICRMPKUBWHWJYGFRQ0AESQGJF2KV1CA3KAQP' # your Foursquare Secret
VERSION = '20181220' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XHHVYHJ5Z54HY4E0YA5SLUVBJPDCXWJBDTS1SC5BYHR3C0YP
CLIENT_SECRET:COCN0HX03VMYICRMPKUBWHWJYGFRQ0AESQGJF2KV1CA3KAQP


#### Let's explore the first neighborhood in our dataframe.

In [20]:
table_df2.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [21]:
neighborhood_latitude = table_df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = table_df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = table_df2.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

In [22]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url1 # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XHHVYHJ5Z54HY4E0YA5SLUVBJPDCXWJBDTS1SC5BYHR3C0YP&client_secret=COCN0HX03VMYICRMPKUBWHWJYGFRQ0AESQGJF2KV1CA3KAQP&v=20181220&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [23]:
results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5c1f0df21ed2194bd671bda1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### Explore Neighborhoods in Manhattan

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=table_df2['Neighbourhood'],
                                   latitudes=table_df2['Latitude'],
                                   longitudes=table_df2['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(103, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Wendy's,43.807448,-79.199056,Fast Food Restaurant
3,Woburn,43.770992,-79.216917,Wendy's,43.807448,-79.199056,Fast Food Restaurant
4,Cedarbrae,43.773136,-79.239476,Wendy's,43.807448,-79.199056,Fast Food Restaurant


In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 1 uniques categories.


##  Analyze Each Neighborhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Fast Food Restaurant
0,"Rouge,Malvern",1
1,"Highland Creek,Rouge Hill,Port Union",1
2,"Guildwood,Morningside,West Hill",1
3,Woburn,1
4,Cedarbrae,1


In [36]:
toronto_onehot.shape

(103, 2)

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Fast Food Restaurant
0,"Adelaide,King,Richmond",1
1,Agincourt,1
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",1
4,"Alderwood,Long Branch",1
5,"Bathurst Manor,Downsview North,Wilson Heights",1
6,Bayview Village,1
7,"Bedford Park,Lawrence Manor East",1
8,Berczy Park,1
9,"Birch Cliff,Cliffside West",1


In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                  venue  freq
0  Fast Food Restaurant   1.0


----Agincourt----
                  venue  freq
0  Fast Food Restaurant   1.0


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                  venue  freq
0  Fast Food Restaurant   1.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0  Fast Food Restaurant   1.0


----Alderwood,Long Branch----
                  venue  freq
0  Fast Food Restaurant   1.0


----Bathurst Manor,Downsview North,Wilson Heights----
                  venue  freq
0  Fast Food Restaurant   1.0


----Bayview Village----
                  venue  freq
0  Fast Food Restaurant   1.0


----Bedford Park,Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant   1.0


----Berczy Park----
                  venue  freq
0  Fast Food Restaurant   1.0


----Birch Cliff,Cliffside West----
                

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
1,Agincourt,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
4,"Alderwood,Long Branch",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
5,"Bathurst Manor,Downsview North,Wilson Heights",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
6,Bayview Village,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
7,"Bedford Park,Lawrence Manor East",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
8,Berczy Park,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
9,"Birch Cliff,Cliffside West",Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant


### Cluster Neighbourhood

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [44]:
toronto_merged = table_df2

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.7845,-79.1605,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.7636,-79.1887,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.771,-79.2169,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Neighbourhood

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
1,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
2,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
3,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
4,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
5,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
6,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
7,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
8,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
9,Scarborough,0,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant,Fast Food Restaurant
